In [1]:

import duckdb


In [2]:
conn = duckdb.connect('respiratory_virus_data.duckdb')

In [3]:
conn.sql("""
         create table if not exists admissions as
         select * from read_csv_auto('Respiratory_Virus_Hospital_Admissions_Over_Time.csv')
         """)

In [3]:
conn.sql("describe admissions")

┌─────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name       │ column_type │  null   │   key   │ default │  extra  │
│         varchar         │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ week_start_date         │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ week_end_date           │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ epi_week                │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ respiratory_virus       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ new_admissions          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ admission_rate_per_100k │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ sf_population_estimate  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ data_updated_at         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ data_as_of    

In [9]:
conn.sql("select * from admissions limit 5")

┌─────────────────┬───────────────┬──────────┬───────────────────┬────────────────┬─────────────────────────┬────────────────────────┬────────────────────────┬────────────────────────┬────────────────────────┐
│ week_start_date │ week_end_date │ epi_week │ respiratory_virus │ new_admissions │ admission_rate_per_100k │ sf_population_estimate │    data_updated_at     │       data_as_of       │     data_loaded_at     │
│      date       │     date      │  int64   │      varchar      │     int64      │         double          │         int64          │        varchar         │        varchar         │        varchar         │
├─────────────────┼───────────────┼──────────┼───────────────────┼────────────────┼─────────────────────────┼────────────────────────┼────────────────────────┼────────────────────────┼────────────────────────┤
│ 2024-12-08      │ 2024-12-14    │       50 │ RSV               │             17 │                    2.03 │                 836321 │ 2025/11/14 08:05:00 AM │ 

In [28]:
conn.sql("""
         select distinct(week_start_date) 
         from admissions
         order by week_start_date desc
         """)

┌─────────────────┐
│ week_start_date │
│      date       │
├─────────────────┤
│ 2025-11-02      │
│ 2025-10-26      │
│ 2025-10-19      │
│ 2025-10-12      │
│ 2025-10-05      │
│ 2025-09-28      │
│ 2025-09-21      │
│ 2025-09-14      │
│ 2025-09-07      │
│ 2025-08-31      │
│     ·           │
│     ·           │
│     ·           │
│ 2024-12-15      │
│ 2024-12-08      │
│ 2024-12-01      │
│ 2024-11-24      │
│ 2024-11-17      │
│ 2024-11-10      │
│ 2024-11-03      │
│ 2024-10-27      │
│ 2024-10-20      │
│ 2024-10-13      │
├─────────────────┤
│     56 rows     │
│   (20 shown)    │
└─────────────────┘

In [10]:
conn.sql("""
         select respiratory_virus, sum(new_admissions) as total_admissions
         from admissions
         group by respiratory_virus
         order by total_admissions desc
         """)

┌───────────────────┬──────────────────┐
│ respiratory_virus │ total_admissions │
│      varchar      │      int128      │
├───────────────────┼──────────────────┤
│ COVID-19          │             1082 │
│ Influenza         │             1044 │
│ RSV               │              410 │
└───────────────────┴──────────────────┘

In [13]:
conn.sql("""
         select respiratory_virus, sum(new_admissions) as total_admissions_in_2025
         from admissions
         where week_start_date between '2025-01-01' and '2025-12-31'
         and week_end_date between '2025-01-01' and '2025-12-31'
         group by respiratory_virus
         order by total_admissions_in_2025 desc
         """)

┌───────────────────┬──────────────────────────┐
│ respiratory_virus │ total_admissions_in_2025 │
│      varchar      │          int128          │
├───────────────────┼──────────────────────────┤
│ COVID-19          │                      927 │
│ Influenza         │                      838 │
│ RSV               │                      265 │
└───────────────────┴──────────────────────────┘

In [21]:
conn.sql("""
         select epi_week, sum(new_admissions) as total_admissions
         from admissions
         group by epi_week
         order by total_admissions desc limit 10
         """)

┌──────────┬──────────────────┐
│ epi_week │ total_admissions │
│  int64   │      int128      │
├──────────┼──────────────────┤
│        5 │              171 │
│        4 │              165 │
│        3 │              149 │
│        6 │              141 │
│        1 │              141 │
│        7 │              139 │
│        2 │              123 │
│       52 │              102 │
│        8 │               95 │
│       34 │               67 │
├──────────┴──────────────────┤
│ 10 rows           2 columns │
└─────────────────────────────┘

In [23]:
conn.sql(""" 
         select respiratory_virus, sum(new_admissions) as total_admissions
         from admissions
         where epi_week = 5
         group by respiratory_virus
         order by total_admissions desc
         """)

┌───────────────────┬──────────────────┐
│ respiratory_virus │ total_admissions │
│      varchar      │      int128      │
├───────────────────┼──────────────────┤
│ Influenza         │              119 │
│ RSV               │               29 │
│ COVID-19          │               23 │
└───────────────────┴──────────────────┘

In [24]:
conn.sql(""" 
         select respiratory_virus, sum(new_admissions) as total_admissions
         from admissions
         where epi_week = 4
         group by respiratory_virus
         order by total_admissions desc
         """)

┌───────────────────┬──────────────────┐
│ respiratory_virus │ total_admissions │
│      varchar      │      int128      │
├───────────────────┼──────────────────┤
│ Influenza         │              117 │
│ RSV               │               34 │
│ COVID-19          │               14 │
└───────────────────┴──────────────────┘

In [25]:
conn.sql(""" 
         select respiratory_virus, sum(new_admissions) as total_admissions
         from admissions
         where epi_week = 3
         group by respiratory_virus
         order by total_admissions desc
         """)

┌───────────────────┬──────────────────┐
│ respiratory_virus │ total_admissions │
│      varchar      │      int128      │
├───────────────────┼──────────────────┤
│ Influenza         │              106 │
│ RSV               │               24 │
│ COVID-19          │               19 │
└───────────────────┴──────────────────┘

In [5]:
conn.sql("""
         select case 
            when epi_week between 49 and 52 then 'Winter'
            when epi_week between 1 and 9 then 'Winter'
            when epi_week between 10 and 22 then 'Spring'
            when epi_week between 23 and 35 then 'Summer'
            when epi_week between 36 and 48 then 'Fall'
            end as season,
            respiratory_virus,
            sum(new_admissions) as total_admissions
        from admissions
        group by season, respiratory_virus
        order by total_admissions desc
         """)

┌─────────┬───────────────────┬──────────────────┐
│ season  │ respiratory_virus │ total_admissions │
│ varchar │      varchar      │      int128      │
├─────────┼───────────────────┼──────────────────┤
│ Winter  │ Influenza         │              881 │
│ Summer  │ COVID-19          │              362 │
│ Fall    │ COVID-19          │              322 │
│ Winter  │ RSV               │              320 │
│ Winter  │ COVID-19          │              223 │
│ Spring  │ COVID-19          │              175 │
│ Spring  │ Influenza         │              113 │
│ Spring  │ RSV               │               49 │
│ Fall    │ Influenza         │               38 │
│ Fall    │ RSV               │               37 │
│ Summer  │ Influenza         │               12 │
│ Summer  │ RSV               │                4 │
├─────────┴───────────────────┴──────────────────┤
│ 12 rows                              3 columns │
└────────────────────────────────────────────────┘

In [29]:
conn.close()